In [ ]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
import os
from langchain.vectorstores import Chroma
from pprint import pprint
from dotenv import load_dotenv

In [ ]:
embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

#db = Chroma.from_documents(docs, embedding_function, persist_directory="./chromaDB")
db = Chroma(persist_directory="./mentalhealthembedings", embedding_function=embedding_function)

In [ ]:
query = "I am feeling depressed because of exams"
docs = db.similarity_search(query)

print(len(docs)) 
docs[0]

In [ ]:
load_dotenv()
huggingFaceAPiKey = os.getenv("HUGGING_API_KEY")

repoID="mistralai/Mistral-7B-Instruct-v0.1"

llmmodel = HuggingFaceHub(repo_id=repoID, model_kwargs={"max_new_tokens": 500, "temperature": 0.4},huggingfacehub_api_token=huggingFaceAPiKey)


template = """You are an AI mental health therapist Eva. You are an expert in psychology and you help people suffering from mental health problems by offering them solutions and making them feel better. Always be polite, kind and respectful. You have a collection of case studies of previous patients and their problems. Whenever possible use the case studies to formulate a similiar response. Generate long and helpful responses.


Case Studies:
================
{context}
================


Question: {human_input}
Helpful Response: """


prompt = PromptTemplate(
    input_variables=["human_input", "context"], template=template
)


chain = load_qa_chain(llm=llmmodel, chain_type="stuff", prompt=prompt, verbose=True)

In [ ]:
query = "I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. What should i do?"
docs = db.similarity_search(query)
print("Done")
response = chain.run(human_input=query, input_documents=docs, return_only_outputs=True)
pprint(response.strip().replace("\xa0", ""))